<a href="https://colab.research.google.com/github/GauriV1/Data-Manifesto-Cookbook/blob/main/Data_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cookbook

## For CS 215, Final
## Professor Wirfs Brock
## Submitted by - Gauri Vaidya



# 1. Cleaning and Transforming Weather Data


We start with a sample weather dataset (project 1) to demonstrate basic Pandas operations: loading data, converting units, and filtering. Suppose we have a CSV of hourly weather measurements (temperature in Fahrenheit, humidity, wind speed, etc.).

We will:
*   Load the data into a Pandas DataFrame.
*   Convert temperatures from Fahrenheit to Celsius (vectorized and via a function).
*   Add a new column indicating if dew is likely (temperature ≤ dew point).
*   Filter rows for a specific condition (e.g., high humidity).

In [ ]:
import pandas as pd

# Sample weather data as a small example (DateTime, Temp_out (°F), Dew_point (°F), Humidity, Wind_speed)
data = {
    "date": ["2025-05-12 00:00", "2025-05-12 01:00", "2025-05-12 02:00", "2025-05-12 03:00"],
    "temp_out": [50.0, 49.5, 49.0, 48.5],
    "dew_point": [48.0, 47.5, 47.0, 46.5],
    "humidity": [85, 87, 88, 90],
    "wind_speed": [5.0, 4.5, 4.8, 5.2]
}
df_weather = pd.DataFrame(data)
df_weather["date"] = pd.to_datetime(df_weather["date"])
print(df_weather.head())


So it would look something like this:

date   temp_out  dew_point  humidity  wind_speed
*  0 2025-05-12  00:00:00      50.0       48.0        85         5.0
*  1 2025-05-12  01:00:00      49.5       47.5        87         4.5
*  2 2025-05-12  02:00:00      49.0       47.0        88         4.8
*  3 2025-05-12  03:00:00      48.5       46.5        90         5.2

Next, we convert temperatures to Celsius using the formula C = (F - 32) * 5/9. We show two methods: vectorized arithmetic and applying a function.


In [ ]:

# Method 1 -  Vectorized arithmetic (fast)
df_weather["temp_c_vector"] = (df_weather["temp_out"] - 32) * 5/9

# Method 2 - Using a function with apply (works per entry)
def f_to_c(f):
    return (f - 32) * 5/9
df_weather["temp_c_apply"] = df_weather["temp_out"].apply(f_to_c)

print(df_weather[["temp_out", "temp_c_vector", "temp_c_apply"]])


Which would give us results like these:

   temp_out  temp_c_vector  temp_c_apply
0      50.0      10.000000      10.000000
1      49.5       9.722222       9.722222
2      49.0       9.444444       9.444444
3      48.5       9.166667       9.166667


Both methods give the same result. The vectorized version (Method 1) is usually faster for large data. We can now add a dew/frost likelihood column: if the air temperature is at or below the dew point, dew is likely. We compare temp_out and dew_point row by row.   

In [ ]:
df_weather["dew_likely"] = df_weather.apply(
    lambda row: "Yes" if row["temp_out"] <= row["dew_point"] else "No", axis=1
)
print(df_weather[["date", "temp_out", "dew_point", "dew_likely"]])


In [ ]:
# Would give us results like these

#                 date  temp_out  dew_point dew_likely
#0 2025-05-12 00:00:00      50.0       48.0         No
#1 2025-05-12 01:00:00      49.5       47.5         No
#2 2025-05-12 02:00:00      49.0       47.0         No
#3 2025-05-12 03:00:00      48.5       46.5         No


In [ ]:
# Finally, we filter the DataFrame to find, for example, readings with humidity above 85%. We can use boolean indexing.

high_humidity = df_weather[df_weather["humidity"] > 85]
print("High-humidity rows:")
print(high_humidity)


#Which would give us results like this:
#High-humidity rows:
#                 date  temp_out  dew_point  humidity  wind_speed  temp_c_vector  temp_c_apply dew_likely
#1 2025-05-12 01:00:00      49.5       47.5        87         4.5       9.722222       9.722222         No
#2 2025-05-12 02:00:00      49.0       47.0        88         4.8       9.444444       9.444444         No
#3 2025-05-12 03:00:00      48.5       46.5        90         5.2       9.166667       9.166667         No

This recipe shows how to clean and transform data: adding new columns, converting units, and filtering rows with Pandas

# 2. Merging and Analyzing Student Course Data

In this recipe, we use example data inspired by project 3. We have two datasets:

*   courses_df: Records of student course enrollments (Student ID, Subject, Course Number, Grade).
*   outcomes_df: Student outcomes (Student ID, Primary Major).

We will merge these tables on Student ID and then analyze which courses were taken by students of a particular major. No data sets will be uploaded to maintain privacy.

In [ ]:
import pandas as pd

# Example data
courses_df = pd.DataFrame({
    "StudentID": [1,1,2,2,3,3,4,4],
    "Subject": ["MATH","ENGL","MATH","BIOL","CHEM","CHEM","MATH","ENGL"],
    "CourseNumber": ["101","101","101","201","101","102","201","101"],
    "Grade": [3.7, 3.3, 3.0, 2.7, 3.3, 3.0, 3.7, 2.7]
})
outcomes_df = pd.DataFrame({
    "StudentID": [1,2,3,4],
    "Major1": ["BIOL","BIOL","CHEM","CHEM"]
})

# Merging on StudentID, keeping only matching records
merged_df = pd.merge(courses_df, outcomes_df, on="StudentID", how="inner")
print("Merged DataFrame:")
print(merged_df)


In [ ]:
# Which would give us something like this

#Merged DataFrame:
#  StudentID Subject CourseNumber  Grade Major1
#0          1    MATH          101    3.7   BIOL
#1          1    ENGL          101    3.3   BIOL
#2          2    MATH          101    3.0   BIOL
#3          2    BIOL          201    2.7   BIOL
#4          3    CHEM          101    3.3   CHEM
#5          3    CHEM          102    3.0   CHEM
#6          4    MATH          201    3.7   CHEM
#7          4    ENGL          101    2.7   CHEM


Now, suppose we want to find out which 1xx courses are most popular among Biology majors (Major1 == "BIOL"). We filter for biology majors and courses numbered in the 100s (the CourseNumber starts with '1' and has length 3).

In [ ]:
# Focus on Biology majors and 1xx courses
bio_df = merged_df[merged_df["Major1"] == "BIOL"].copy()
bio_df["FullCourse"] = bio_df["Subject"] + bio_df["CourseNumber"]
bio_df_1xx = bio_df[bio_df["CourseNumber"].str.startswith("1")]
course_counts = bio_df_1xx["FullCourse"].value_counts()
print("1xx courses taken by BIOL majors:")
print(course_counts)


In [ ]:
# Would give us something like this

#1xx courses taken by BIOL majors:
#MATH101    2
#ENGL101    1
#dtype: int64


The output shows that MATH101 and ENGL101 were taken by biology majors (with counts). We could further analyze these grades or visualize them, but for brevity we just count courses here. This recipe demonstrates merging tables and filtering for analysis.


# 3. Clustering City Data (K-Means)

We use a dataset inspired by project 5 (city parks data). Suppose we have cities with attributes "Population" and "Investment in Parks (dollars)". We want to cluster similar cities. We use K-Means clustering from scikit-learn.

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

# Example city data
city_data = pd.DataFrame({
    "City": ["A","B","C","D","E","F","G","H"],
    "Population": [50000, 120000, 30000, 150000, 80000, 200000, 25000, 110000],
    "Invest_dollars": [500000, 1200000, 400000, 1500000, 700000, 2000000, 300000, 1000000]
})
print("City data:")
print(city_data)

In [ ]:
# Which would give us something like

#. City data:
#  City  Population  Invest_dollars
#0    A       50000          500000
#1    B      120000         1200000
#2    C       30000          400000
#3    D      150000         1500000
#4    E       80000          700000
#5    F      200000         2000000
#6    G       25000          300000
#7    H      110000         1000000


We perform K-Means clustering on the (Population, Investment) features. We choose 3 clusters (for example).

In [ ]:
features = city_data[["Population", "Invest_dollars"]]
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(features)
city_data["Cluster"] = clusters
print("\nCluster assignments:")
print(city_data[["City","Cluster"]])


In [ ]:
#Which would give something like this

#Cluster assignments:
#  City  Cluster
#0    A        2
#1    B        1
#2    C        2
#3    D        0
#4    E        2
#5    F        0
#6    G        2
#7    H        1


Each city is assigned to a cluster 0, 1, or 2. We can inspect the cluster centers and sizes:

In [ ]:
# Cluster centers and counts
print("\nCluster centers (Population, Investment):")
print(kmeans.cluster_centers_)
print("\nCounts per cluster:")
print(city_data["Cluster"].value_counts().sort_index())

In [ ]:
"""Cluster centers (Population, Investment):
[[150000. 1500000.]
 [115000. 1100000.]
 [ 46875.  537500.]]

Counts per cluster:
0    2
1    2
2    4"""

In this example, cluster 0 centers on the largest cities, cluster 2 on the smallest, and cluster 1 in between. This recipe illustrates clustering: grouping cities by similar features. In practice, we would visualize these clusters (e.g., scatter plot) and interpret them. Clustering since is an unsupervised method; I would like to try different algorithms (e.g., hierarchical clustering) on the same data and compare results.

# 4. Splitting Data into Training and Testing Sets

This recipe shows how I embed the Train vs Test efficiency evaluation directly into Streamlit dashboard (from app.py on final project).

Split historical data into “in-sample” (train) and “out-of-sample” (test) portions

Fit an ARIMA model on the train set

Compute an efficiency metric for both sets

Display the results as a bar chart side-by-side

Why it matters: Transparent model evaluation is crucial—this gives end users instant feedback on how well forecasting pipeline generalizes.

In [ ]:
with tab4:
    st.subheader(f"Train vs Test Efficiency — {selected_ticker}")

    # 1. Prepare the in-sample (train) data
    #    All dates before the forecast window (May 1–5, 2025)
    hist_train = clean_df[
        (clean_df['Ticker'] == selected_ticker) &
        (clean_df['Date'] < forecast_start)
    ]

    # 2. Fit ARIMA(3,1,5) on the train series
    arima_model = ARIMA(hist_train['Close'], order=(3,1,5)).fit()

    # 3. Compute residuals and in-sample efficiency
    resid     = hist_train['Close'] - arima_model.fittedvalues
    train_eff = (1 - resid.abs() / hist_train['Close']).mean()

    # 4. Reuse the avg_eff from the “Model Efficiency” tab as out-of-sample (test) efficiency
    test_eff = avg_eff

    # 5. Assemble a small DataFrame for plotting
    df_eff = pd.DataFrame({
        'Period': ['Train', 'Test'],
        'Efficiency (%)': [train_eff * 100, test_eff * 100]
    })

    # 6. Plot a bar chart with Plotly
    fig3 = go.Figure(go.Bar(
        x=df_eff['Period'],
        y=df_eff['Efficiency (%)'],
        text=df_eff['Efficiency (%)'].round(2),
        textposition='auto'
    ))
    fig3.update_layout(
        title="Train vs Test Efficiency (%)",
        yaxis={'range': [0, 100]}
    )
    st.plotly_chart(fig3, use_container_width=True)


1. . Selecting the Training Data
We begin by isolating all historical data prior to our forecast window (May 1–5, 2025). By filtering clean_df on the chosen ticker and dates earlier than forecast_start, we ensure the model only “sees” past data during fitting.

2. Fitting the ARIMA Model
Next, we fit an ARIMA(3,1,5) model on this in-sample series. This replicates the exact configuration used for forecasting—calling

3. 3. Computing In-Sample Efficiency
Once the model is trained, we calculate daily residuals as nd convert these into an efficiency score per day:

daily efficiency
= 1 - mod (resid)/actual

Averaging across all days yields train_eff, a single value between 0 and 1 that quantifies how well the model fits its own training data.

4. Defining Test Efficiency
For the test set, we repurpose the average efficiency previously calculated on the May 1–5, 2025 forecast window (avg_eff). This captures how well the ensemble generalizes to truly unseen data.

5. Preparing Data for Visualization
We assemble a small two-row DataFrame with the labels “Train” and “Test” alongside their respective efficiencies converted to percentages. This simple table is the basis for our comparative plot.

6. Displaying the Comparison
Finally, we render a Plotly bar chart in Streamlit: one bar for the train score and one for the test score, each labeled with its exact percentage. By placing this inside a dedicated “Train vs Test” tab, users immediately see both how well the model learned the historical patterns and how robust it remains on future dates.

Altogether, this recipe turns dry numeric evaluation into an interactive, visually intuitive dashboard component—strengthening trust in forecasting pipeline by making its limitations and strengths crystal clear.

# Conclusion

The Data Cookbook integrates four key recipes—processing and cleaning weather data, merging and analyzing student course records, clustering city park investments, and integrating a train/test efficiency measure into a Streamlit dashboard—each of which includes actual code, brief explanations, and test outputs. By drawing on datasets from semester projects, these recipes demonstrate practical techniques I would like to return to again and again: vectorized transformations versus function-based approaches, DataFrame merges and filters, unsupervised clustering with scikit-learn, and transparent model evaluation inside an interactive app.

At its base, there is a **Data Science Manifesto** of my values: making black-box AI understandable, taking on Data Feminism to make equity and justice primary concerns, pushing for open-source and open-access in order to encourage shared accountability, and a culture of curiosity and exploration. Both the manifesto and cookbook together constitute an interdisciplinary toolkit—not just for fixing problems, but for doing so in ethical, transparent, and innovative ways.

As I would go on to work on future data science projects—research, internships, or personal projects—this notebook hopefully will be my quick reference to repeat patterns and reminder of the principles that inform good data practice. Keep adding to it: new recipes, refining existing ones, and let the manifesto evolve as my skills develop with time. In that manner, this cookbook will be a living book—a present to my future self and everyone else who treads my data-driven path. :)